In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime

# Importing 3 Dataframes

In [2]:
df_parcel = pd.read_csv('../Data/raw/EXTR_Parcel.csv', encoding = "ISO-8859-1")

In [ ]:
df_res = pd.read_csv('../Data/raw/EXTR_ResBldg.csv')

In [ ]:
df_real = pd.read_csv('../Data/raw/EXTR_RPSale.csv', encoding = "ISO-8859-1")

# Filtering Real Sales Dataframe

In [5]:
#set to 
df_real_home = df_real.loc[(df_real.PropertyType.isin([2,3,11])) & (df_real.SalePrice > 0)]

In [ ]:
#Creating Column to combine Major and minor called UniqueID 
df_real_home['UniqueID'] = (df_real_home['Major'].astype(str)+ df_real_home['Minor'].astype(str)).astype(int)

In [ ]:
df_real_home['DocumentDate'] = pd.to_datetime(df_real_home['DocumentDate'], format="%m/%d/%Y")

In [8]:
#dropping duplicated values of unique id and keeping the most recent sale date
df_real_home = df_real_home.sort_values('DocumentDate').drop_duplicates('UniqueID', keep='last')

# Filtering Parcel and Resident dataframe

In [9]:
df_res['UniqueID'] = (df_res['Major'].astype(str)+ df_res['Minor'].astype(str)).astype(int)
df_parcel['UniqueID'] = (df_parcel['Major'].astype(str)+ df_parcel['Minor'].astype(str)).astype(int)

In [10]:
df_parcel = df_parcel[df_parcel.PropType == 'R']

In [11]:
df_parcel = df_parcel.drop_duplicates(subset = 'UniqueID', keep = 'first')

In [12]:
df_res = df_res.drop_duplicates(subset = 'UniqueID', keep = 'first')

# Making UniqueID column the index for all the dataframes

In [13]:
df_parcel.set_index('UniqueID', drop = True, inplace = True)

In [14]:
df_res.set_index('UniqueID', drop = True, inplace = True)

In [15]:
df_real_home.set_index('UniqueID', drop = True, inplace = True)

# Concatenating the Dataframes 

In [20]:
test_df = pd.concat([df_real_home, df_parcel.reindex(df_real_home.index)], axis=1)

In [22]:
df_combined = pd.concat([test_df, df_res.reindex(test_df.index)], axis=1)

In [32]:
df_combined = df_combined.loc[:,~df_combined.columns.duplicated()]

In [83]:
df_combined.columns.tolist()

['ExciseTaxNbr',
 'Major',
 'Minor',
 'DocumentDate',
 'SalePrice',
 'RecordingNbr',
 'Volume',
 'Page',
 'PlatNbr',
 'PlatType',
 'PlatLot',
 'PlatBlock',
 'SellerName',
 'BuyerName',
 'PropertyType',
 'PrincipalUse',
 'SaleInstrument',
 'AFForestLand',
 'AFCurrentUseLand',
 'AFNonProfitUse',
 'AFHistoricProperty',
 'SaleReason',
 'PropertyClass',
 'SaleWarning',
 'PropName',
 'PlatName',
 'Range',
 'Township',
 'Section',
 'QuarterSection',
 'PropType',
 'Area',
 'SubArea',
 'SpecArea',
 'SpecSubArea',
 'DistrictName',
 'LevyCode',
 'CurrentZoning',
 'HBUAsIfVacant',
 'HBUAsImproved',
 'PresentUse',
 'SqFtLot',
 'WaterSystem',
 'SewerSystem',
 'Access',
 'Topography',
 'StreetSurface',
 'RestrictiveSzShape',
 'InadequateParking',
 'PcntUnusable',
 'Unbuildable',
 'MtRainier',
 'Olympics',
 'Cascades',
 'Territorial',
 'SeattleSkyline',
 'PugetSound',
 'LakeWashington',
 'LakeSammamish',
 'SmallLakeRiverCreek',
 'OtherView',
 'WfntLocation',
 'WfntFootage',
 'WfntBank',
 'WfntPoorQual

In [79]:
df_final = df_combined[['DocumentDate','SalePrice', 'PropertyType', 'PrincipalUse', 'SaleReason', 'PropertyClass', 'SaleWarning', 'Area', 'SqFtLot', 'LakeWashington',
 'LakeSammamish','SmallLakeRiverCreek','OtherView','WfntLocation','WfntFootage','WfntBank','WfntPoorQuality','WfntRestrictedAccess','WfntAccessRights','WfntProximityInfluence','TidelandShoreland', 'TrafficNoise',
 'AirportNoise','PowerLines','OtherNuisances', 'SqFtTotLiving','SqFtDeck','SqFtOpenPorch', 'Condition' ]]

In [80]:
df_final['year'] = pd.DatetimeIndex(df_final['DocumentDate']).year

<ipython-input-80-b02a7a58dda0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['year'] = pd.DatetimeIndex(df_final['DocumentDate']).year


In [81]:
df_2019final = df_final[df_final.year == 2019]

In [82]:
df_2019final.to_json('../Data/combined.json')